# Introdução ao Controle
O controle é uma disciplina fundamental da engenharia que lida com a gestão dinâmica dos sistemas para que eles se comportem de maneira desejada. A essência do controle está em monitorar e ajustar automaticamente a operação de sistemas, desde simples dispositivos mecânicos até complexas redes de comunicação e sistemas robóticos.

O princípio básico é regular as variáveis de um sistema mantendo-as próximas a um valor desejado ou setpoint. Isso é feita através do calculo do erro entre o valor desejado e o valor medido da variável controlada, e então aplicando ações corretivas para minimizar esse erro.

## Tipos de Controle
Existem dois tipos principais de controle: o controle em malha aberta e o controle em malha fechada:

1. Controle em Malha Aberta: Não há feedback do processo. As ações de controle são baseadas em um conjunto pré-definido de instruções que não se alteram em resposta ao estado atual do sistema. Um exemplo simples seria um forno elétrico programado para ficar ligado por um tempo determinado, independentemente da temperatura real dentro do forno.

2. Controle em Malha Fechada (ou Controle com Feedback): Usa o feedback do estado atual do sistema para tomar decisões de controle. Este tipo de controle é mais adaptativo e pode corrigir desvios em relação ao setpoint. Um exemplo seria adicionar um termostato ao forno elétrico para desligá-lo quando a temperatura desejada for atingida.

## Controle Proporcional (P)
Um dos métodos mais simples e amplamente utilizados no controle em malha fechada é o controle proporcional. Este método ajusta a saída do controlador de forma proporcional à diferença (erro) entre o valor desejado (setpoint) e o valor medido da variável controlada. O coeficiente que determina a relação entre o erro e a ação de controle é conhecido como ganho proporcional (K_p). Portanto uma ação de controle proporcional é dada pela seguinte equação:

```python
Ação de Controle = K_p * erro
```

Na esqueção acima, podemos ver que a ação de controle é proporcional ao erro. Se o erro for grande, a ação de controle será grande. Se o erro for pequeno, a ação de controle será pequena. Portanto no exemplo do forno elétrico, ao ligar o forno frio, o erro será grande e a ação de controle será grande, aumentando a potência do forno e elevando a temperatura rapidamente. À medida que a temperatura se aproxima do setpoint, o erro diminui reduzindo a potência do forno, evitando que a temperatura ultrapasse o setpoint drasticamente.

## Resposta da Ação de Controle
Outros conceitos importantes no controle são a resposta da ação de controle e o conceito de estabilidade. A resposta da ação de controle é a reação do sistema à ação de controle. Se a resposta for muito lenta, o sistema pode não atingir o setpoint ou pode crescer indefinidamente, levando o sistema a um estado instável. Se a resposta for muito rápida, o sistema pode ultrapassar o setpoint e oscilar em torno dele. Portanto, é importante ajustar o ganho proporcional para obter uma resposta rápida e estável.

# Exemplo de Controle Proporcional
A seguir, implementamos um exemplo simples de controle proporcional em Python. Neste exemplo, simulamos um sistema de controle de um drone que tenta manter a altitude constante para uma dada entrada de altitude, setpoint. Neste exemplo, você pode ajustar o ganho proporcional (K_p) e a resposta do sistema para visualizar como o controle proporcional afeta a resposta do sistema.

A figura abaixo ilustra o sistema:

![Drone](figs/drone.png)

Durante seus experimentos, tente responder às seguintes perguntas:
1. O que acontece se o ganho proporcional for muito baixo? E se for muito alto?
2. Qual a relação entre o ganho proporcional e a resposta do sistema para o momento em que o sistema nunca atinge o setpoint, ou seja, o sistema é instável?

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider

# Parâmetros da simulação
setpoint = 10.0  # Altitude desejada em metros

def controle_proporcional(setpoint, altitude_atual, Kp):
    """
    Calcula a velocidade do rotor necessária para ajustar a altitude do drone com base no erro entre a altitude desejada e a atual.

    Args:
        setpoint (float): Altitude alvo desejada em metros.
        altitude_atual (float): Altitude atual do drone em metros.
        Kp (float): Ganho proporcional.
        
    Returns:
        float: Ajuste da velocidade do rotor (W).
    """
    erro = setpoint - altitude_atual
    W = Kp * erro  # Velocidade do rotor para ajustar a altitude
    return W

def resposta_do_sistema(W, altitude_atual, tempo_de_atualizacao):
    """
    Simula a resposta da altitude do drone ao ajuste da velocidade do rotor.

    Args:
        W (float): Velocidade do rotor ajustada pelo controlador.
        altitude_atual (float): Altitude atual do drone em metros.
        tempo_de_atualizacao (float): Intervalo de tempo para cada atualização em segundos.
        
    Returns:
        float: Altitude atualizada.
    """
    # Mudança de altitude como resposta direta ao ajuste da velocidade do rotor
    nova_altitude = altitude_atual + W * tempo_de_atualizacao
    return nova_altitude

def executar_simulacao(Kp, tempo_de_atualizacao, altitude_inicial):
    """
    Executa a simulação de ajuste de altitude para o drone, permitindo a interação do usuário com parâmetros do sistema.

    Args:
        Kp (float): Ganho proporcional.
        tempo_de_atualizacao (float): Intervalo de atualização em segundos.
        altitude_inicial (float): Altitude inicial do drone em metros.
    """
    tempos = np.arange(0, 10 + tempo_de_atualizacao, tempo_de_atualizacao)  # Simular por 10 segundos
    altitudes = [altitude_inicial]
    altitude_atual = altitude_inicial
    
    for t in tempos[1:]:
        W = controle_proporcional(setpoint, altitude_atual, Kp)
        altitude_atual = resposta_do_sistema(W, altitude_atual, tempo_de_atualizacao)
        altitudes.append(altitude_atual)
    
    plt.figure(figsize=(10, 5))
    plt.plot(tempos, altitudes, label='Altitude do Drone')
    plt.plot(tempos, [setpoint] * len(tempos), 'r--', label='Altitude Alvo')
    plt.title(f'Controle de Altitude do Drone com Kp={Kp}, Tempo de Atualização={tempo_de_atualizacao}s, Altitude Inicial={altitude_inicial}')
    plt.xlabel('Tempo (segundos)')
    plt.ylabel('Altitude (metros)')
    plt.legend()
    plt.grid(True)
    plt.show()

# Widget interativo
interact(executar_simulacao, 
         Kp=FloatSlider(value=0.0, min=0.0, max=20.0, step=0.1, description='Kp (Ganho):'),
         tempo_de_atualizacao=FloatSlider(value=0.25, min=0.05, max=1.0, step=0.05, description='Tempo de Atualização (s):'),
         altitude_inicial=FloatSlider(value=0.0, min=0, max=20.0, step=1.0, description='Altitude Inicial (m):'))


interactive(children=(FloatSlider(value=0.0, description='Kp (Ganho):', max=20.0), FloatSlider(value=0.25, des…

<function __main__.executar_simulacao(Kp, tempo_de_atualizacao, altitude_inicial)>